In [1]:
!pip install phate
!pip install umap-learn
!pip install pyod
!pip install cubist
!pip install catboost
!pip install sklearn-rvm
!pip install dataheroes
!pip install https://github.com/JamesRitchie/scikit-rvm/archive/master.zip

  Obtaining dependency information for phate from https://files.pythonhosted.org/packages/04/7e/85ea7d79b1150b08edcf5aac5bc6b601074f2b5cc43e8f3936b10d6e3b0a/phate-1.0.11-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
  Obtaining dependency information for scprep>=0.11.1 from https://files.pythonhosted.org/packages/2a/d2/72a06c97668b07ef703ffdc392cbd5ce004733d14978f6ae2ae71a09df41/scprep-1.2.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.6/602.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | / done
  Created wheel for pyod: filename=pyod-1.1.1-py3-none-any.whl size=190074 sha256=2593731fe2edcd713709b1a3c18835a86addcf8fd4b45503948c9680b645

In [2]:
import numpy as np
import pandas as pd
import math
import scipy
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn import impute
from sklearn import model_selection
from sklearn.svm import SVR
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.metrics import r2_score

import cubist
import numpy as np
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, StackingRegressor, RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeCV, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from skrvm import RVR

pd.DataFrame.iteritems = pd.DataFrame.items


In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/task-1/X_test.csv
/kaggle/input/task-1/y_train.csv
/kaggle/input/task-1/X_train.csv


In [4]:
def read_data(X_train_path, y_train_path, X_test_path):
    X_train = np.genfromtxt(X_train_path, delimiter=",")
    y_train = np.genfromtxt(y_train_path, delimiter=",")
    X_test = np.genfromtxt(X_test_path, delimiter=",")
    return X_train, y_train, X_test

In [5]:
# Read file
X_train = pd.read_csv(
    "/kaggle/input/task-1/X_train.csv",
    sep = ",",
    index_col = "id"
)

y_train = pd.read_csv(
    "/kaggle/input/task-1/y_train.csv",
    sep = ",",
    index_col = "id"
)

X_test = pd.read_csv(
    "/kaggle/input/task-1/X_test.csv",
    sep = ",",
    index_col = "id"
)
ids_test = X_test.index

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1212, 832)
(1212, 1)
(776, 832)


In [6]:
def mp(x):
  if x == 1:
    return True
  else:
    return False

In [7]:
qt = QuantileTransformer(output_distribution="normal", random_state=np.random.RandomState(304))

impute_data = X_train.fillna(X_train.median())

clf = LocalOutlierFactor()
outliers = clf.fit_predict(impute_data)
liers = outliers == 1
print(liers)
labels = y_train[liers]
impute_data = impute_data[liers]
selecter = SelectKBest(f_regression, k=200)
impute_data = selecter.fit_transform(impute_data, labels.values.flatten())
final_x = qt.fit_transform(impute_data)

[ True  True  True ...  True  True  True]


In [8]:
impute_test = X_test.fillna(X_train.median())
impute_test = selecter.transform(impute_test)
final_test = qt.transform(impute_test)

In [9]:
print(type(final_x))

<class 'numpy.ndarray'>


In [10]:
X_train = np.array(final_x)
X_test = np.array(final_test)
y_train = np.array(labels).ravel()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)


(1170, 200)
(1170,)
(776, 200)


In [11]:

def get_model(method = 1):
    if method == 1:
        model = GaussianProcessRegressor(kernel=RationalQuadratic(alpha=3.5), random_state=42)
    elif method == 2:
        estimators = [
            ('xgb', XGBRegressor(n_estimators=1000, eta=0.1, colsample_bytree=0.9,
                                 gamma=0.5, learning_rate=0.1, max_depth=8, min_child_weight=10)),  # 0.520819
            ('extratree', ExtraTreesRegressor(n_estimators=1000, random_state=0)),
            ('adaboost', AdaBoostRegressor(n_estimators=1000, random_state=0)),
            #('lgbm', LGBMRegressor(learning_rate=0.01, max_bin=1000, num_iterations=3000, verbose=0)),  # 0.508068
            ('svr_rbf', SVR(C=10, coef0=0.01, degree=3, gamma='scale',kernel='rbf')),  # 0.515443
            # ('mn', KNeighborsRegressor(n_neighbors=15, p=2, weights='distance')),  # 0.441597
            ('rvm', RVR(alpha=1e-06)),
            ('cat', CatBoostRegressor(verbose=False)),
            ('gp', GaussianProcessRegressor(kernel=RationalQuadratic(alpha=3.5), random_state=42)),
            ('cubist', cubist.Cubist()),
        ]
        model = StackingRegressor(estimators=estimators,
                                  final_estimator=RandomForestRegressor(n_estimators=100, random_state=42))
    return model

In [12]:
def get_splits(X_train: np.array, nfolds: int = 10):
    kf = KFold(n_splits=nfolds, random_state=42, shuffle=True)
    return kf.split(X_train)


def main():

    nfolds = 10
    splits = get_splits(X_train, nfolds)

    print("\nModels and folds.")

    r2, rmse_total = 0, 0
    models = []
    for i, (train_index, test_index) in enumerate(splits):
        model = get_model(1)
        print(X_train[train_index].shape)
        print(y_train[train_index].shape)
        print(X_train[test_index].shape)
        model.fit(X_train[train_index], y_train[train_index])
        pred = model.predict(X_train[test_index])
        score = r2_score(y_train[test_index], pred)
        r2 += score

        rmse = mean_squared_error(y_train[test_index], pred, squared=False)
        rmse_total += rmse

        models.append(model)

        print(f"Fold {i} R2 score: {score}, RMSE: {rmse}")

    print(f"\nAvg R2: {r2 / nfolds}, RMSE: {rmse_total / nfolds}")

    print("\nTrained.")

    from_folds = True
    if from_folds:
        pred = models[0].predict(X_test)
        pred_train = models[0].predict(X_train)
        for model in models[1:]:
            pred += model.predict(X_test)
            pred_train += model.predict(X_train)

        pred = pred / nfolds
        pred_train = pred_train / nfolds

    else:
        model = get_model(1)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        pred_train = model.predict(X_train)

    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)
    print(f"\nr2 ov train data (overfit): {r2_train}, RMSE: {rmse_train}")

    res = np.column_stack((ids_test, pred))
    np.savetxt("out.csv", res, fmt=['%1i', '%1.4f'], delimiter=",", header="id,y", comments='')

if __name__ == "__main__":
    main()


Models and folds.
(1053, 200)
(1053,)
(117, 200)
Fold 0 R2 score: 0.6179603386077046, RMSE: 6.390013962803514
(1053, 200)
(1053,)
(117, 200)
Fold 1 R2 score: 0.5125680811602439, RMSE: 6.416149224923412
(1053, 200)
(1053,)
(117, 200)
Fold 2 R2 score: 0.5345933232180644, RMSE: 5.689110316838292
(1053, 200)
(1053,)
(117, 200)
Fold 3 R2 score: 0.5987914503794642, RMSE: 5.785871510154444
(1053, 200)
(1053,)
(117, 200)
Fold 4 R2 score: 0.5739899796010866, RMSE: 6.082509942749503
(1053, 200)
(1053,)
(117, 200)
Fold 5 R2 score: 0.6163014466347938, RMSE: 5.450664490624041
(1053, 200)
(1053,)
(117, 200)
Fold 6 R2 score: 0.29511179921642783, RMSE: 8.194511057859268
(1053, 200)
(1053,)
(117, 200)
Fold 7 R2 score: 0.5821668745499102, RMSE: 6.179316045491805
(1053, 200)
(1053,)
(117, 200)
Fold 8 R2 score: 0.49067283662998395, RMSE: 6.830115939702071
(1053, 200)
(1053,)
(117, 200)
Fold 9 R2 score: 0.4972222330818, RMSE: 7.174862709871647

Avg R2: 0.5319378363079479, RMSE: 6.4193125201017995

Trained